Template notebook for one type of model.

In [34]:
def reload_helper_functions():
    """Reload helper_functions.py if it is already imported."""
    if "helper_functions" in sys.modules:
        importlib.reload(sys.modules["helper_functions"])
    else:
        import helper_functions

In [48]:
import importlib
import sys
reload_helper_functions()

# General imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Model specific imports
from sklearn.tree import DecisionTreeClassifier

In [29]:
# Load data
df_train = pd.read_csv("../../data/train_data_enriched.csv", index_col="Claim Identifier")
df_test = pd.read_csv("../../data/test_data_enriched.csv",index_col="Claim Identifier")

C:\Users\Louis\AppData\Local\Temp\ipykernel_15984\2503440325.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv("../../data/train_data_enriched.csv", index_col="Claim Identifier")
C:\Users\Louis\AppData\Local\Temp\ipykernel_15984\2503440325.py:3: DtypeWarning: Columns (2,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv("../../data/test_data_enriched.csv",index_col="Claim Identifier")


In [17]:
# ------------------------

In [36]:
selected_features = ['Accident Date', 'Age at Injury', 'Alternative Dispute Resolution', 'Attorney/Representative', 'Average Weekly Wage', 'C-3 Date', 'Carrier Name', 'Carrier Type', 'County of Injury',
                      'COVID-19 Indicator', 'District Name', 'First Hearing Date', 'Gender', 'IME-4 Count', 'Industry Code', 'Medical Fee Region', 'WCIO Cause of Injury Code', 'WCIO Nature of Injury Code', 
                      'WCIO Part Of Body Code', 'Number of Dependents', 'Accident Year', 'Accident Month', 'Accident Day', 'Accident DayOfWeek', 'C-2 Date DSA', 'C-3 Date DSA',
                         'Age at Injury Category', # 'Accident Date_missing', 'First Hearing Date_missing', 'C-3 Date_missing', 'Assembly Date_missing', 'C-2 Date_missing',
                        'Carrier Claim Category', 'Body Section']

# Select Categorical features
selected_cat_features = ['Alternative Dispute Resolution', 'Attorney/Representative', 'Carrier Name', 'Carrier Type', 'County of Injury',
                        'COVID-19 Indicator', 'District Name', 'Gender', 'Industry Code', 'Medical Fee Region', 'WCIO Cause of Injury Code', 'WCIO Nature of Injury Code',
                        'WCIO Part Of Body Code', 'Carrier Claim Category', 'Body Section'] # based on RFE

# Select numerical features
selected_num_features = ['Age at Injury', 'Average Weekly Wage','IME-4 Count','Number of Dependents',
                         'Accident Year','Accident Month', 'Accident Day', 'Accident DayOfWeek',
                         'C-2 Date DSA', 'C-3 Date DSA'] # based on RFE

In [45]:
le = LabelEncoder()
#Label inconding our target variable 
df_train["Claim Injury Type"] = le.fit_transform(df_train["Claim Injury Type"])

In [46]:
X = df_train.drop(["Claim Injury Type"], axis = 1)
y = df_train["Claim Injury Type"]

In [49]:
dt_results = helper_functions.cv_scores(DecisionTreeClassifier(), X[selected_features], y, selected_num_features, selected_cat_features)
dt_results
# look at results

Age at Injury           float64
Average Weekly Wage     float64
IME-4 Count             float64
Number of Dependents    float64
Accident Year           float64
Accident Month          float64
Accident Day            float64
Accident DayOfWeek      float64
C-2 Date DSA            float64
C-3 Date DSA            float64
dtype: object


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:Accident Date: object, Alternative Dispute Resolution: object, C-3 Date: object, Carrier Name: object, Carrier Type: object, County of Injury: object, District Name: object, First Hearing Date: object, Gender: object, Medical Fee Region: object, Age at Injury Category: object, Carrier Claim Category: object, Body Section: object

In [ ]:
# Test prediction
    # Choose best model from KFold above
    # Train model on whole train and predict test data
    submission = helper_functions.test_prediction(XGBClassifier(),X[selected_features],y,data_test)
    submission